In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [2]:
import tensorflow as tf
def swish(x):
    return x*tf.nn.sigmoid(x)

In [3]:
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import EarlyStopping,CSVLogger,ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.convnext import ConvNeXtXLarge

In [4]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

In [5]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

In [6]:
x_train=tf.keras.applications.convnext.preprocess_input(x_train)
x_test=tf.keras.applications.convnext.preprocess_input(x_test)

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [8]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [9]:
base_model_xlarge=ConvNeXtXLarge(
    include_top=False,
    weights='imagenet',
    input_shape=(64,64,3)
)

In [10]:
base_model_xlarge.trainable=False

In [11]:
model=models.Sequential([
    base_model_xlarge,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ convnext_xlarge (Functional)         │ (None, 2, 2, 2048)          │     348,147,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 350,935,140 (1.31 GB)

 Trainable params: 2,783,588 (10.62 MB)

 Non-trainable params: 348,151,552 (1.30 GB)

In [12]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [13]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [14]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [15]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [16]:
import time
start_time=time.time()

In [17]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 161s 133ms/step - accuracy: 0.1239 - loss: 4.3884 - val_accuracy: 0.6246 - val_loss: 1.6072
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 137s 114ms/step - accuracy: 0.4029 - loss: 2.5566 - val_accuracy: 0.6789 - val_loss: 1.2653
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - accuracy: 0.4745 - loss: 2.1174 - val_accuracy: 0.7041 - val_loss: 1.1232
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - accuracy: 0.5210 - loss: 1.8805 - val_accuracy: 0.7187 - val_loss: 1.0452
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - accuracy: 0.5435 - loss: 1.7632 - val_accuracy: 0.7307 - val_loss: 0.9895
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - accuracy: 0.5633 - loss: 1.6505 - val_accuracy: 0.7342 - val_loss: 0.9570
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 135s 115ms/step - accuracy: 0.5794 - loss: 1.6009 - val_accuracy: 0.7427 - val_loss: 0.9291
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 136s 116ms/step - ac

In [18]:
head_training_time=time.time()-start_time

In [19]:
base_model_xlarge.trainable=True

In [20]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [21]:
start_fine_tune=time.time()

In [22]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 380s 304ms/step - accuracy: 0.7660 - loss: 0.8250 - val_accuracy: 0.8639 - val_loss: 0.4622
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 323s 263ms/step - accuracy: 0.8375 - loss: 0.5621 - val_accuracy: 0.8782 - val_loss: 0.4185
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 308s 263ms/step - accuracy: 0.8729 - loss: 0.4383 - val_accuracy: 0.8874 - val_loss: 0.3886
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 307s 262ms/step - accuracy: 0.8891 - loss: 0.3852 - val_accuracy: 0.8897 - val_loss: 0.3805
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 307s 262ms/step - accuracy: 0.9017 - loss: 0.3315 - val_accuracy: 0.8937 - val_loss: 0.3726
Epoch 6/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 308s 263ms/step - accuracy: 0.9206 - loss: 0.2726 - val_accuracy: 0.8958 - val_loss: 0.3718
Epoch 7/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 308s 263ms/step - accuracy: 0.9336 - loss: 0.2303 - val_accuracy: 0.9002 - val_loss: 0.3587
Epoch 8/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 306s 261ms/step - ac

In [23]:
fine_tune_time=time.time()-start_fine_tune

In [24]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 68ms/step


In [25]:
test_loss,test_accuracy=model.evaluate(x_test,y_test,verbose=1)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 67ms/step - accuracy: 0.8985 - loss: 0.3705

Test Accuracy: 89.62%
Total training time: 5892.26 seconds


In [26]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 23s 59ms/step - accuracy: 0.9025 - loss: 0.3468
Validation accuracy: 90.0160%


In [27]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for ConvNext Extra Large:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for ConvNext Extra Large:
Accuracy: 0.8962
Precision: 0.8983
Recall: 0.8962
F1-Score: 0.8963


In [28]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["ConvNext Extra Large"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [29]:
print(results)

                  Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  ConvNext Extra Large         0.8962              0.90016   0.369273   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0             2724.456932           3167.799352     350935140  


In [30]:
results.to_csv("ConvNext XLarge.csv",index=False)
print("\nResults saved to ConvNext XLarge.csv")


Results saved to ConvNext XLarge.csv
